In [8]:
import requests
import json


In [37]:
response = requests.get('https://static.openfoodfacts.org/data/taxonomies/additives.json')

if response.status_code == 200:
    data = response.json()
    for key in data:
        try:
            additive = data.get(key)
            # print(additive)
            additive_classes = additive.get('additives_classes').get('en').replace('en:', '').replace(' ', '').split(',')
            
            print(additive_classes)
        except AttributeError:
            continue
else:
    print(f"Une erreur s'est produite lors du télechargement du fichier")

['colour']
['stabiliser', 'thickener']
['carrier', 'emulsifier', 'humectant', 'stabiliser', 'thickener']
['colour']
['emulsifier', 'humectant', 'sequestrant', 'stabiliser', 'thickener']
['thickener']
['colour']
['emulsifier', 'stabiliser', 'thickener']
['emulsifier', 'stabiliser', 'thickener']
['humectant', 'sequestrant', 'stabiliser', 'sweetener', 'thickener']
['colour']
['carrier', 'emulsifier', 'humectant', 'stabiliser', 'thickener']
['colour']
['preservative']
['emulsifier', 'stabiliser', 'thickener']
['emulsifier', 'stabiliser', 'thickener']
['emulsifier', 'stabiliser']
['emulsifier', 'humectant', 'sequestrant', 'stabiliser', 'thickener']
['colour']
['sweetener']
['emulsifier', 'stabiliser', 'thickener']
['emulsifier']
['colour']
['preservative']
['colour']
['colour']
['stabiliser', 'thickener']
['antioxidant']
['thickener']
['emulsifier']
['colour']
['colour']
['stabiliser', 'thickener']
['emulsifier', 'stabiliser', 'thickener']
['antioxidant']
['humectant']
['colour']
['emulsifi

In [44]:
from selenium import webdriver

from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup
import time


options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)  


driver.get("https://www.additifs-alimentaires.net/additifs.php")

time.sleep(10)  

html = driver.page_source

soup = BeautifulSoup(html, "html.parser")

table = soup.select_one("table.tpi tbody")

additives_list = []
additionnal_additives_data = requests.get('https://static.openfoodfacts.org/data/taxonomies/additives.json')
data_aditional = additionnal_additives_data.json()

for row in table.find_all('tr'):
    additive_code_ = row.select_one('td.colCode').get_text(strip=True).lower()
    print(additive_code_)
    additive_additional_dict = data_aditional.get(f"en:{additive_code_}")
    print(f"en:{additive_code_}")
    print(additive_additional_dict)
    try:
        additive_classes = additive_additional_dict.get('additives_classes').get('en').replace('en:', '').replace(' ', '').split(',')
    except AttributeError:
            additive_classes = []
    print(additive_classes)
    additive_dict = {
        'additive_code': row.select_one('td.colCode').get_text(strip=True),
        'names': row.select_one('td.colNom').get_text(strip=True).split(', '),
        'danger': row.select_one('td.colDanger').get_text(strip=True),
        'additive_classes': additive_classes,
    }
    print(additive_dict)
    # print(row.get_text(separator="\n", strip=True))
    print("\n")
    additives_list.append(additive_dict)
# print(table.get_text(separator="\n", strip=True))
print(len(additives_list))
driver.quit()


e100
en:e100
{'efsa_evaluation': {'en': 'Scientific Opinion on the re-evaluation of curcumin -E 100- as a food additive'}, 'efsa_evaluation_url': {'en': 'http://dx.doi.org/10.2903/j.efsa.2010.1679'}, 'anses_additives_of_interest': {'en': 'yes'}, 'name': {'hr': 'E100 - kurkumin', 'nb': 'E100 - kurkumin', 'lt': 'E100 - kurkuminas', 'nl': 'E100 - curcumine', 'fr': 'E100 - curcumine', 'kn': 'E100 - ಕರ್ಕ್ಯುಮಿನ್', 'et': 'E100 - kurkumiin', 'es': 'E100 - curcumina', 'pl': 'E100 - kurkumina', 'he': 'E100 - כורכומין', 'ar': 'E100 - كركومين', 'mt': 'E100 - kurkumina', 'en': 'E100 - Curcumin', 'bs': 'E100 - kurkumin', 'ro': 'E100 - curcumină', 'lv': 'E100 - kurkumīns', 'cs': 'E100 - kurkumin', 'fa': 'E100 - کورکومین', 'sk': 'E100 - kurkumín', 'sl': 'E100 - kurkumin', 'ja': 'E100 - クルクミン', 'ta': 'E100 - குர்க்குமின்', 'hu': 'E100 - kurkumin', 'el': 'E100 - κουρκουμινη', 'ca': 'E100 - curcumina', 'ru': 'E100 - куркумины', 'ko': 'E100 - 커큐민', 'bg': 'E100 - куркумин', 'uk': 'E100 - куркумін', 'sv': '

In [45]:
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)  

complete_additives_list = []
for additive in additives_list:
    code = additive['additive_code']

    url = f"https://www.additifs-alimentaires.net/{code}.php#toxic"
    
    try: 
        driver.get(url)
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        texts = soup.find_all("td", class_="colonne2")

        description = texts[0].get_text(strip=True)
        description_avancee = texts[2].get_text(strip=True)
        remarques = texts[4].get_text(strip=True)
        additive['description'] = description
        additive['description_avancee'] = description_avancee
        additive['remarques'] = remarques
        print(additive)
        complete_additives_list.append(additive)
    except Exception as e:
        print(f"Erreur lors de la récupération de la page pour {code}: {e}")
        continue


    

{'additive_code': 'E100', 'names': ['Curcumines', 'Jaune de curcumine'], 'danger': 'Ne pas abuser', 'additive_classes': ['colour'], 'description': "La curcumine est le pigment principal jaune-orange de la plante herbacée curcuma (Curcuma longa, ouSafran des Indes)  [21a]. Elle peut être isolée par distillation à la vapeur, ou extraite parsolvantsd'extraction chimiques [24,9], comme c'est le cas dans l'Union Européenne [15f].", 'description_avancee': 'La curcumine E100 est généralement décrite sans effet secondaire connu [1,2,4,9,9\',9",10,22,28] sauf réactions allergiques rares (?dèmes de Quincke, urticaires aigües, etc.) [9,22,23,29].Mais la curcumine naturelle, connue pour ses effets bénéfiques, ne doit pas être confondue avec la curcumine du Codex alimentarius ou E100 qui, bien que d\'origine naturelle peut contenir jusque 30% d\'aluminium (E173), et des traces de solvants dans ses résidus de productionVoir sec°. Description > Définitions officielles > Résidus.Cette curcumine de l\'

In [46]:
import json

with open('../additives_complet_data.json', 'wb') as file : 
        file.write(json.dumps(complete_additives_list, ensure_ascii=False, indent=4).encode('utf-8'))
print("Fichier JSON créé avec succès.")

Fichier JSON créé avec succès.


In [ ]:


from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client["Nutriwiser"]
collection = db["additives"]

for additive in complete_additives_list:
    document = {
        'additive_code': additive['additive_code'],
        'names': additive['names'],
        'danger': additive['danger']
    }
    print(document)


    collection.insert_one(document)



{'additive_code': 'E100', 'names': ['Curcumines', 'Jaune de curcumine'], 'danger': 'Ne pas abuser'}
{'additive_code': 'E101', 'names': ['Riboflavine', "Riboflavine-5'-phosphate", 'Vitamine B2'], 'danger': 'Ne pas abuser'}
{'additive_code': 'E102', 'names': ['Tartrazine'], 'danger': 'Très toxique'}
{'additive_code': 'E104', 'names': ['Jaune de quinoléine', 'Jaune de quinoline'], 'danger': 'Très toxique'}
{'additive_code': 'E105', 'names': ['Jaune solide'], 'danger': 'Très toxique'}
{'additive_code': 'E107', 'names': ['Jaune 2G'], 'danger': 'Toxique'}
{'additive_code': 'E110', 'names': ['Jaune orangé S', 'Jaune soleil FCF'], 'danger': 'Très toxique'}
{'additive_code': 'E111', 'names': ['Orange GGN', 'Alpha-naphthol orange'], 'danger': 'Très toxique'}
{'additive_code': 'E120', 'names': ['Cochenille', 'Acide Carminique', 'Carmins'], 'danger': 'Douteux'}
{'additive_code': 'E121', 'names': ['Rouge citrus no. 2 (ex- Orseille', 'Orcéine)'], 'danger': 'Très toxique'}
{'additive_code': 'E122', '